In [1]:
import pandas as pd, numpy as np
pd.options.display.float_format = '{:,.4f}'.format
import plotly.express as px; from plotly import graph_objects as go
import datetime as dt; ts = dt.datetime.now(); ts_str = ts.strftime("%Y_%m_%d_%H_%M_%S"); print(ts_str)

2020_09_08_14_29_00


In [2]:
sql = 'SELECT * FROM [MPlanProd].[dbo].[View_Transaction_SFT_6049]'
con = (
    r'DRIVER=ODBC Driver 17 for SQL Server;'
    r'SERVER=172.31.228.90;'
    r'DATABASE=MPlanProd;'
    r'Trusted_Connection=no;'
    r'Uid=DataEng;'
    r'Pwd=Data@2020;'
)

In [3]:
%%time
import pyodbc
cxn = pyodbc.connect(con)
df1 = pd.read_sql(sql, cxn)
print(df1.shape, ', '.join(df1.columns))

(284594, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 44.9 s


In [4]:
df = df1.copy()
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','ตาก','บางกระทุ่ม', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]

In [5]:
col_keep = ['Zone', 'Sloc', 'Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton']
df[col_keep].to_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', index=False)
#df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
#df['dt'] = pd.to_datetime(df['dt'].astype(str) + ' ' + df['Time'].astype(str))

In [ ]:
#df.SiteName.value_counts()#.sum()

In [ ]:
#df.Sloc.value_counts()#.sum()

# Cumulative

In [ ]:
ym, ymi, ymc = {}, {}, {}
months = [dt.date(2008, i, 1).strftime('%b') for i in range(1,13)]

In [ ]:
for t in ['Purchase_price_thb_ton','MillWeight']:
    ym[t] = df.groupby(['Year','Month'])[t].sum().reset_index()
    ymi[t]= ym[t][ym[t].Year>=2016].reset_index(drop=True)
    ymc[t]= ymi[t].groupby('Year').cumsum().merge(ymi[t], left_index=True, right_index=True, suffixes=('_c',''))

In [ ]:
ymp = ymc['Purchase_price_thb_ton'].merge(ymc['MillWeight'])
ymp['Price']  = ymp['Purchase_price_thb_ton'] / ymp['MillWeight']
ymp['Price_c']= ymp['Purchase_price_thb_ton_c'] / ymp['MillWeight_c']

In [ ]:
ymp.Year = ymp.Year.astype('category')
ymp.Month = ymp.Month.astype('category')
ymp

In [ ]:
for t in ['Purchase_price_thb_ton','MillWeight']:
    if t == 'MillWeight': cols = ['MillWeight', 'MillWeight_c']
    else: cols = ['Purchase_price_thb_ton', 'Purchase_price_thb_ton_c']
    for col in cols:
        fig = go.Figure()
        for year in ymc[t].Year.unique():
            Y  = ymc[t].loc[ymc[t].Year==year]
            fig.add_trace(go.Scatter(x=months, y=Y[col], mode="markers+text+lines", textposition="middle right",
                                     text=['' for i in range(len(Y)-1)]+[str(year)]))
        fig.update_layout(showlegend=False, title=col)
        fig.show()

In [ ]:
for col in ['Price', 'Price_c']:
    fig = go.Figure()
    for year in ymp.Year.unique():
        Y  = ymp.loc[ymp.Year==year]
        fig.add_trace(go.Scatter(x=months, y=Y[col],
                                 mode="markers+text+lines", textposition="middle right",
                                 text=['' for i in range(len(Y)-1)]+[str(year)]))
    fig.update_layout(showlegend=False, title=col)
    fig.show()

# Supply Estimation

In [ ]:
px.scatter(ymp.loc[(ymp.Year==2019)|(ymp.Year==2020)], x='MillWeight', y='Price', color='Year', hover_name='Month')

In [ ]:
px.scatter(ymp.loc[ymp.Month==9], x='MillWeight', y='Price', color='Year')

In [ ]:
px.scatter(ymp, x='MillWeight', y='Price', color='Year')

# Export

In [ ]:
SlocDICT = dict(df.groupby('SiteName')['Sloc'].max())
SlocDICT = dict([(value, key) for key, value in SlocDICT.items()])
ZoneDICT = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

In [ ]:
# WorkCentreName, 'Time'
ck=['Zone','Sloc','SiteName','Ticket_no', 'Year', 'Month', 'Day', 'MillWeight','Price_ton', 'Purchase_price_thb_ton']
df = df[ck]

In [ ]:
wp = {}
for s in SlocDICT.keys():
    df_ = df.loc[df.Sloc==s].groupby(['Year', 'Month', 'Day']).sum().reset_index()
    df_['Price_Daily'] = df_['Purchase_price_thb_ton']/df_['MillWeight']
    df_.drop('Price_ton', axis=1 ,inplace=True)
    wp[s] = df_.to_json(orient = 'records')

In [ ]:
import json
with open('Sloc.json', 'w') as o: json.dump(SlocDICT, o)
with open('Zone.json', 'w') as o: json.dump(ZoneDICT, o)
with open('wp.json', 'w') as o: json.dump(wp, o)

In [ ]:
wpm = {}
with open('wp.json', 'r') as o: wp = json.load(o)
for s in SlocDICT.keys():
    wp[s]          = pd.read_json(wp[s], orient = 'records')
    wp[s]['dt']    = pd.to_datetime(wp[s][['Year', 'Month', 'Day']])
    #wp[s].drop(['Year', 'Month', 'Day'], axis=1, inplace=True)
    wpm[s]         = wp[s].groupby(['Year','Month']).sum().reset_index().drop('Price_Daily', axis=1)
    wpm[s]['Price_Monthly'] = wpm[s]['Purchase_price_thb_ton']/wpm[s]['MillWeight']
    wpm[s]['Day']  = 1   
    wpm[s]['dt']   = pd.to_datetime(wpm[s][['Year', 'Month', 'Day']])
    wpm[s]['Sloc'] = s

# Test

In [ ]:
wp[s]

In [ ]:
wpms = {}
for s in SlocDICT.keys():
    sen        = wpm[s][['MillWeight','Price_Monthly']].pct_change().apply(lambda x: round(x, 5))*100
    sen['sen'] = sen['MillWeight'] / sen['Price_Monthly']
    sen        = sen.replace(np.inf, 1000).replace(-np.inf, -1000).clip(-100, 100)
    wpms[s]    = sen.merge(wpm[s]['dt'], left_index=True, right_index=True)
    wpms[s]['Sloc'] = s

In [ ]:
wpmj, wpmsj = {}, {};
wpmsjz = pd.DataFrame()
for z in ZoneDICT.keys():
    a, b = pd.DataFrame(), pd.DataFrame()
    for s in ZoneDICT[z]:
        a = a.append(wpm[s])
        b = b.append(wpms[s])
    wpmj[z] = a
    wpmsj[z] = b
    wpmsjz = wpmsjz.append(wpmsj[z])

In [ ]:
tl = np.sort(wpmsjz.dt.unique())[::-1]
ss = wpmsjz.Sloc.unique()
dft= pd.DataFrame()
for t in tl:
    dfs = pd.DataFrame(ss, columns=['Sloc'])
    dfs['dt'] = t
    dft = dft.append(dfs)
wpmsjzm = wpmsjz.merge(dft, left_on=['dt','Sloc'], right_on=['dt','Sloc'], how='outer')


# Plot

In [ ]:
len(px.scatter(wpmj[z], x='dt', y='Price_Monthly', size='MillWeight', color='Sloc', size_max=60).data)

In [ ]:
px.line(wpms[s], x='dt', y='sen')

In [ ]:
fig = go.Scatter(x=wpm[s]['dt'].values, y=wpm[s]['Price_Monthly'].values,
                 marker=dict(size=wpm[s]['MillWeight'].values))
fig